In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pyplot as plt
import colorcet as cc
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
import sys
from dash import Dash, html
import dash_cytoscape as cytov
import igraph as ig
from IPython.display import Image

sys.path.append("/Users/jlq293/Projects/Study-1-Bibliometrics/src/main_path/")

from MainPathReader import MainPathReader
from MainPathDataAssigner import MainPathDataAssigner
from MainPathPlotterStatic import MainPathPlotterStatic
from MainPathPlotterInteractive import MainPathPlotterInteractive

# load and prep data


In [8]:
params = "alpha0.3_k20_res0.006"
p = f"../data/06-clustered-df/{params}.pkl"
df = pd.read_pickle(p)

path_to_main_path = (
    "../data/08-main-paths/3. Key-Route Global Main Path [100-150] of N2 (169).net"
)
mp_reader = MainPathReader(
    path_to_main_path,
    node_attributes=[
        "eid",
    ],
)
Gmp = mp_reader.get_graph()
print(list(Gmp.nodes(data=True))[0])

DiGraph with 169 nodes and 203 edges
('1', {'label': 'Magnussen_1982', 'x': '0.9534', 'y': '0.6944', 'size': '0.5000', 'shape': 'ellipse', 'eid': '2-s2.0-0019996341'})


In [16]:
attr_to_assign = [
    "title",
    "citedby_count",
    "doi",
    "year",
    f"cluster_{params}",
    f"centrality_{params}",
]

data_assigner = MainPathDataAssigner(Gmp, df, attr_to_assign)
Gmp = data_assigner.process_mp(attr_to_remove=["x", "y", "size", "shape"])
# rename to 'cluster'
nx.set_node_attributes(Gmp, nx.get_node_attributes(Gmp, f"cluster_{params}"), "cluster")
nx.set_node_attributes(
    Gmp, nx.get_node_attributes(Gmp, f"centrality_{params}"), "centrality"
)

# remove old cluster attribute
# Attribute to remove
attributes_to_remove = [f"cluster_{params}", f"centrality_{params}"]
# Loop through nodes and delete the attribute
for node in Gmp.nodes():
    for attr in attributes_to_remove:
        if attr in Gmp.nodes[node]:
            del Gmp.nodes[node][attr]


print(list(Gmp.nodes(data=True))[0])
clusters = nx.get_node_attributes(Gmp, f"cluster").values()

print(f"Nr of clusters in Gmp: {len(set(clusters))}")

('1', {'label': 'Magnussen_1982', 'eid': '2-s2.0-0019996341', 'title': 'Treatment of myoclonic syndromes with paroxetine alone or combined with 5‐HTP', 'citedby_count': '13', 'doi': '10.1111/j.1600-0404.1982.tb04525.x', 'year': '1982', 'cluster': '57', 'centrality': '0.053737004306467086'})
Nr of clusters in Gmp: 31


# static plot


In [10]:
def convert_to_igraph(G):
    return ig.Graph.from_networkx(G)


def attribute_adapter(g):
    g.vs["auth_year"] = [v["label"] for v in g.vs]
    g.vs["label"] = [
        f"{int(v['_nx_name'])-1},{v['cluster_alpha0.3_k10_res0.002']}" for v in g.vs
    ]
    print(g.vs[0])
    return g


def prepare_layout(g):
    layout = g.layout("kamada_kawai")
    return layout


def prep_ig(G):
    g = convert_to_igraph(G)
    g = attribute_adapter(g)
    layout = prepare_layout(g)
    return g, layout

In [11]:
g, layout = prep_ig(Gmp)

# reverse the layout
layout.rotate(180)


def change_position(layout, node, x_change, y_change):
    x = layout[node][0] + x_change
    y = layout[node][1] + y_change
    layout[node] = (x, y)
    return layout


layout = change_position(layout, 20, -0.2, -0.1)
#
# for i in range(0, 70):
#    layout = change_position(layout, i, 1.20, 0)


# Plot with margins
plot = ig.plot(
    g,
    layout=layout,
    bbox=[2000, 3500],
    labels=True,
    vertex_size=50,
    font_size=10,
    vertex_color=colors,
    vertex_label=[str(node) for node in g.vs["label"]],
    arrow_size=0.0001,
    margin=100,
)

# Save or display the plot
filename = "/Users/jlq293/Projects/Study-1-Bibliometrics/output/figures/main-paths/graph_plot.png"
plot.save(filename)
plot

KeyError: 'Attribute does not exist'